In [ ]:
!pip install tensorflow --upgrade 

In [ ]:
#Required Libraries
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow import *
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras.applications import *
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Copy and paste the training and testing folder in training and testing variable
training_folder=r"/kaggle/input/leaf-disease-detection-dataset/dataset/train"
testing_folder=r"/kaggle/input/leaf-disease-detection-dataset/dataset/test"

In [ ]:
# Find the Number of classess
number_of_classes=len(os.listdir(training_folder))
print(f"Number of classes: {number_of_classes}")

In [ ]:
#Find the Number of images in Training folder
total_number_of_images_in_training=0
folder_path=os.listdir(training_folder)
for i,folder in enumerate(folder_path):
    number_of_images=len(os.listdir(training_folder+'/'+folder))
    total_number_of_images_in_training+=number_of_images
    print(f"{i} , In {folder} folder contains {number_of_images} images.")

print(f"Total Number of Images in Training Folder: {total_number_of_images_in_training}")


In [ ]:
#Find the Number of images in Testing folder
total_number_of_images_in_testing=0
folder_path=os.listdir(testing_folder)
for i,folder in enumerate(folder_path):
    number_of_images=len(os.listdir(testing_folder+'/'+folder))
    total_number_of_images_in_testing+=number_of_images
    print(f"{i}, In {folder} folder contains {number_of_images} images.")

print(f"Total Number of Images in Testing Folder: {total_number_of_images_in_testing}")

In [ ]:
#To Get the filename with pathname and folder name for label for Training Folder
file_path=[] # File name
label=[]  # folder name
for subfolder in os.listdir(training_folder):
    sub_folder_path=os.path.join(training_folder,subfolder)
    if os.path.isdir(sub_folder_path):
        for file_name in os.listdir(sub_folder_path):
            path=os.path.join(sub_folder_path,file_name)
            if os.path.isfile(path):
                file_path.append(path)
                label.append(subfolder)

In [ ]:
# To create a Dataframe for the training data
data={
    'file_path':file_path,
    'label':label
}
train_df=pd.DataFrame(data)


In [ ]:
#Check the Training data
train_df.head()

In [ ]:
# Shape of the Training Data
train_df.shape

In [ ]:
#To Get the filename with pathname and folder name for label in Testing folder
file_path=[] # File name
label=[]  # folder name
for subfolder in os.listdir(testing_folder):
    sub_folder_path=os.path.join(testing_folder,subfolder)
    if os.path.isdir(sub_folder_path):
        for file_name in os.listdir(sub_folder_path):
            path=os.path.join(sub_folder_path,file_name)
            if os.path.isfile(path):
                file_path.append(path)
                label.append(subfolder)

In [ ]:
# To create a Dataframe for the testing data
data={
    'file_path':file_path,
    'label':label
}
test_df=pd.DataFrame(data)


In [ ]:
#Check the Testing data
test_df.head()

In [ ]:
# Shape of the Testing Data
test_df.shape

In [ ]:
#To initialize the values
width,height,depth=256,256,3
batch_size=32
class_mode="categorical"
scale=True
rescale=None
if scale:
    rescale=1./255

In [ ]:
#Training Data Preprocessing
train_datagen=ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
                                 )

In [ ]:
# Test Data Preprocessing
test_datagen=ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
#Training Data
training_data=train_datagen.flow_from_directory(
    training_folder,
    target_size=(width,height),
    batch_size=batch_size,
    class_mode=class_mode,
    subset="training"
    )

In [ ]:
#Validation Data
validation_data=train_datagen.flow_from_directory(
    training_folder,
    target_size=(width,height),
    batch_size=batch_size,
    class_mode=class_mode,
    subset="validation"
    )

In [ ]:
#Testing Data
testing_data=test_datagen.flow_from_directory(
    testing_folder,
    target_size=(width,height),
    batch_size=batch_size,
    class_mode=class_mode
)

In [ ]:
true_labels = testing_data.classes

In [ ]:
true_labels

In [ ]:
#CNN Layers
model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3),input_shape=(width,height,depth), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(number_of_classes,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
history=model.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=2,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
cnn_loss, cnn_accuracy=model.evaluate(testing_data)
print("Testing Accuracy:",cnn_accuracy)

In [ ]:
model.save("cnn.h5")

In [ ]:
model=load_model("cnn.h5")

In [ ]:
#y_pred_probs_cnn = model.predict(testing_data)

In [ ]:
#y_pred_cnn = np.argmax(y_pred_probs_cnn, axis=1)

In [ ]:
#report_cnn = classification_report(true_labels, y_pred_cnn)
#print(report_cnn)

In [ ]:
#accuracy_cnn=accuracy_score(true_labels, y_pred_cnn)
#accuracy_cnn

In [ ]:
# Train CNN architectures 


In [ ]:
#Xception
base_model_exception = Xception(weights='imagenet', include_top=False, input_shape=(width, height, depth))


In [ ]:
x = base_model_exception.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust units and activation as needed
predictions = Dense(number_of_classes, activation='softmax')(x)

model_exception = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model_exception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model_exception.fit(training_data, steps_per_epoch=int(len(training_data)/10),epochs=20, batch_size=32,validation_data=validation_data, validation_steps=int(len(validation_data)/10), callbacks=[early_stopping])

In [ ]:
exception_loss, exception_accuracy = model_exception.evaluate(testing_data)
print(f"Testing Accuracy: {exception_accuracy}")

In [ ]:
from keras.models import save_model

In [ ]:
save_model(model_exception,"exception.h5")

In [ ]:
from keras.models import load_model

In [ ]:
model=load_model("exception.h5")

In [ ]:
# VGG16

In [ ]:
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(width, height, depth))


In [ ]:
x = base_model_vgg16.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust units and activation as needed
predictions = Dense(number_of_classes, activation='softmax')(x)

model_vgg16 = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model_vgg16.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
history_vgg16=model_vgg16.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
vgg16_loss, vgg16_accuracy = model_vgg16.evaluate(testing_data)
print(f"Testing Accuracy: {vgg16_accuracy}")

In [ ]:
#VGG19

In [ ]:
base_model_vgg19 = VGG19(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet')
 # Freeze the base model

In [ ]:
x = base_model_vgg19.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust units and activation as needed
predictions = Dense(number_of_classes, activation='softmax')(x)

model_vgg19 = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model_vgg19.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history_vgg19=model_vgg19.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
vgg19_loss, vgg19_accuracy = model_vgg19.evaluate(testing_data)
print(f"Testing Accuracy: {vgg19_accuracy}")

In [ ]:
#Resnet50

In [ ]:
base_model_resnet50 = ResNet50(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet',
)

In [ ]:
x = base_model_resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust units and activation as needed
predictions = Dense(number_of_classes, activation='softmax')(x)

model_resnet50 = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model_resnet50.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history_resnet50=model_resnet50.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
resnet50_loss,resnet50_accuracy = model_resnet50.evaluate(testing_data)
print(f"Testing Accuracy: {resnet50_accuracy}")

In [ ]:
#InceptionV3

In [ ]:
base_model_inceptionv3 = InceptionV3(
    input_shape=(width, height, depth),
    include_top=False,
    weights='imagenet')  # Freeze the base model

In [ ]:
x = base_model_inceptionv3.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust units and activation as needed
predictions = Dense(number_of_classes, activation='softmax')(x)

model_inceptionv3 = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model_inceptionv3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history_inceptionv3=model_inceptionv3.fit(
    training_data,
    steps_per_epoch=int(len(training_data)/10),
    epochs=20,
    validation_data=validation_data,
    validation_steps=int(len(validation_data)/10),
    callbacks=[early_stopping]
)

In [ ]:
inceptionv3_loss,inceptionv3_accuracy = model_inceptionv3.evaluate(testing_data)
print(f"Testing Accuracy: {inceptionv3_accuracy}")

In [ ]:
def preprocess_image(img_path):
    # Preprocess the image
    #img_path = '/kaggle/input/leaf-disease-detection-dataset/images_for_test/AppleCedarRust2.JPG'
    img = image.load_img(img_path, target_size=(256, 256))  # Adjust target_size as per your model's input size
    # Preprocess the image
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image if required by your model
    return img_array

In [ ]:
def predict_image(img_path):
    # Predict the class
    processed_image=preprocess_image(img_path)
    predictions = model_exception.predict(processed_image)
    # Interpret the output
    predicted_class = np.argmax(predictions, axis=1)
    #print(f'Predicted class: {predicted_class}')
    return predicted_class

In [ ]:
class_names=sorted(os.listdir("/kaggle/input/leaf-disease-detection-dataset/dataset/train"))
print(class_names)

In [ ]:
true=[]
pred=[]

In [ ]:
testing_image="/kaggle/input/leaf-disease-detection-dataset/images_for_test"
for images in os.listdir(testing_image):
    img_path=testing_image+'/'+images
    predictions=predict_image(img_path)
    print(images)
    true.append(images)
    result=class_names[predictions[0]]
    print(result)
    pred.append(result)

In [ ]:
for i in range(len(true)):
    print(true[i], pred[i])

In [ ]:
import tensorflow as tf
from tensorflow import keras 
print(f"Keras version: {keras.__version__}")
print(f"TensorFlow version: {tf.__version__}")